# 데이터 검증 방법


## Hold Out 교차검증(train_test_split)
- train / test
- train 데이터가 적으면 성능의 분산이 커짐(underfitting)
- 반대로 train 데이터가 커지면 성능의 신뢰도 하락(overfitting)

보통 8 : 2 / 9 : 1로 데이터를 나누어 사용한다.

In [25]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split,\
KFold, StratifiedKFold, cross_val_score, LeaveOneOut, ShuffleSplit, GroupKFold
import pandas as pd

In [14]:
data = pd.DataFrame(load_breast_cancer().data, columns = load_breast_cancer().feature_names)
data['target'] = load_breast_cancer().target

In [15]:
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [16]:
### target 데이터의 분포에 맞추어 8:2로 분리(stratify = data['target']이기 때문)
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data['target'], test_size = .2, random_state = 42, stratify = data.target)

In [17]:
X_train.shape, X_test.shape

((455, 30), (114, 30))

***
## K-겹 교차 검증

- 홀드아웃에 비해 훈련 세트의 분할에 덜 민감한 성능 추정을 얻을 수 있다.
- 중복을 허락하지 않고 훈련 데이터를 k개의 폴드로 랜덤하게 나눈 뒤 k-1개의 폴드로 모델을 훈련하고 나머지 하나의 폴드로 성능을 평가한다.
- 훈련 데이터가 적다면 폴드 갯수를 늘리는 것이 좋다.
- K값이 커지면 훈련 데이터가 여러번 반복 사용되고 모델 성능을 평균하여 일반화 성능을 추정할 때 더 낮은 편향을 만든다.
- 회귀에는 일반적인 K-겹 교차 검증을 사용한다.

In [18]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

In [19]:
# cv = cross_val_score(model, X, y, cv = kf, scoring = ~~)

***
## Leave-One-Out-Cross-Validation

- 1개의 데이터만 검증에 사용하고 n-1개의 데이터를 모두 학습에 사용한다.
- 모든 샘플에 대해서 한번씩 test를 하기 때문에 무작위성이 존재하지 않고 더 많은 데이터를 학습에 사용한다.
- 컴퓨팅 자원 소모가 크다.

In [20]:
loo = LeaveOneOut()

In [21]:
#cv = cross_val_score(model, X, y, cv = loo, scoring = ~~)
# 모든 데이터를 검증데이터로 활용한다. 데이터가 500개이면 test 500번 수행

***
## ShuffleSplit(임의분할 교차검증)

- 훈련과 검증데이터의 크기를 유연하게 조절해야할 때 유용하다.
- 반복 횟수를 훈련 데이터나 검증 데이터의 크기와 독립적으로 조절해야할 때 유용.
- 정수로 train_size, test_size 지정하면 개수로 float형태로 지정하면 비율로 추출함.

In [22]:
ss = ShuffleSplit(test_size = .4, train_size = .6, n_splits = 10)

In [23]:
#cv = cross_val_score(model, X, y, cv = ss, scoring = ~~)

***
## GroupKFold

- 각 데이터를 종속변수가 아닌 변수(그룹형 변수)를 기준으로 KFold분할 
- 기준 변수 지정 필요
- 분류의 경우 각 클래스들의 특징을 살릴 수 있음

In [26]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
gk = GroupKFold(n_splits=2)

In [ ]:
#cv = cross_val_score(model, X, y, cv = gk, scoring = ~~)

***

## TimeSeriesSplit

- 훈련 데이터를 키워나가는 방식
- 일정한 시간 간격이 있는 데이터의 경우 활용

In [29]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits = 5)
print(tscv)
for train, test in tscv.split(X):
    print("%s %s" % (train, test))

TimeSeriesSplit(max_train_size=None, n_splits=5)
[0] [1]
[0 1] [2]
[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]
